In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

add_paths = [
        '/Users/jsegovia/cvx'
        '../Datasets/',
        '../Auxiliary_Functions/',
        '../Cov_Shift_Gen_Functions/',
        '../DWGCS/',
    ]
# Add paths
import sys
for add_path in add_paths:
    sys.path.append(add_path)

from CovShiftGen import CovShiftGen
from DWGCS import DWGCS

dataset_normalize = np.genfromtxt('../Datasets/Blood_normalize.csv', delimiter=',')
d = dataset_normalize.shape[1]
X = dataset_normalize[:,:d-1]

nbrs = NearestNeighbors(n_neighbors=50, algorithm='auto').fit(X)
distances, _ = nbrs.kneighbors(X)
sigma_ = np.mean(distances[:, 49])

print("The sigma value is,", sigma_)

The sigma value is, 0.7490798130732806


In [2]:
feature = 1
[Train_Set,Test_Set,n,t] = CovShiftGen.Features(dataset_normalize, feature)
xtr = Train_Set[:,:d-1]
ytr = Train_Set[:,d-1:].astype(int)
xte = Test_Set[:,:d-1]
yte = Test_Set[:,d-1:].astype(int)

In [4]:
class Mdl:
    def __init__(self,intercep,deterministic,feature_map,labels,lambda0,loss,sigma_):
        self.intercep = intercep
        self.deterministic = deterministic
        self.feature_map = feature_map
        self.labels = labels
        self.lambda0 = lambda0 
        self.loss = loss
        self.sigma_ = sigma_

# DKMM 0-1-loss
D = 1.0 / np.square(1.0 - (np.arange(0.0, 1.0, 0.1)))
RU_Dwgcs = np.zeros(len(D))
error_Dwgcs = np.zeros(len(D))

Dwgcs =[]
for l in range(len(D)):
    mdl_aux = Mdl(False,True,'linear',2,0,'0-1',sigma_)
    mdl_aux.D = D[l]
    Dwgcs.append(mdl_aux)
    Dwgcs[l] = DWGCS.DWKMM(Dwgcs[l],xtr,xte)
    Dwgcs[l] = DWGCS.parameters(Dwgcs[l],xtr,ytr,xte)
    Dwgcs[l] = DWGCS.learning(Dwgcs[l],xte)
    RU_Dwgcs[l] = Dwgcs[l].RU

RU_best_Dwgcs_01 = np.min(RU_Dwgcs)
position = np.argmin(RU_Dwgcs)
Dwgcs[l] = DWGCS.prediction(Dwgcs[l],xte,yte)
error_best_Dwgcs = Dwgcs[l].error

Prueba1 = DWGCS.DWKMM(Prueba1,xtr,xte)
print("The beta weights are,")
print(Prueba1.beta_.T)
print("The alpha weights are,")
print(Prueba1.alpha_.T)
print("The optimization value of the DW-KMM is", Prueba1.min_DWKMM)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jul 20 08:53:22 PM: Your problem has 6 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 20 08:53:22 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 20 08:53:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 20 08:53:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 20 08:53:22 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jul 20 08:53:22 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

NameError: name 'Prueba1' is not defined

In [ ]:
Prueba1 = DWGCS.parameters(Prueba1,xtr,ytr,xte)

print("El vector tau es,")
print( Prueba1.tau_)
print("El vector lambda es,")
print( Prueba1.lambda_)

In [ ]:
Prueba1 = DWGCS.learning(Prueba1,xte)

print("El vector mu es,")
print( np.ravel(Prueba1.mu_))
print("The value of the MRC optimization problem using weights beta obtained using DW-GCS is", Prueba1.RU)

In [ ]:
Prueba1 = DWGCS.prediction(Prueba1,xte,yte)

print("El error de clasificación es",)
print( Prueba1.error)